<a href="https://colab.research.google.com/github/tiadoherty/tia_doherty_code_sample/blob/main/Tia_Doherty_Code_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random
import string
import numpy as np
from datetime import datetime, timedelta

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#logic demonstrated here: using loops, appending data, working with different file types

april_file_path = '/content/drive/MyDrive/Code Sample Data/april_tickets.json'
may_file_path = '/content/drive/MyDrive/Code Sample Data/may_tickets.json'

april_data = []
may_data=[]

#convert json into pandas df; I typically use Spark for this but pandas is more friendly for local files
with open(april_file_path, 'r') as file:
    for line in file:
        april_data.append(pd.read_json(line, typ='series'))

with open(may_file_path, 'r') as file:
    for line in file:
        may_data.append(pd.read_json(line, typ='series'))

df_april = pd.DataFrame(april_data)
df_may = pd.DataFrame(may_data)

<ipython-input-4-9b38b20675f7>:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  april_data.append(pd.read_json(line, typ='series'))
<ipython-input-4-9b38b20675f7>:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  april_data.append(pd.read_json(line, typ='series'))
<ipython-input-4-9b38b20675f7>:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  april_data.append(pd.read_json(line, typ='series'))
<ipython-input-4-9b38b20675f7>:12: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  april_data.append(pd

In [5]:
#quick validation on the two dataframes:
df_april.head()

,ticketId,createDate,problemSummary,status,accountNumber,region,escalated,frt,assigned_to,resolutionDetails,endtime
0,KYR361R47B,2024-04-02 18:15:23,Late Payment Fee Dispute,CLOSED,282416533844,NORTHEAST,True,2024-04-02 22:15:23,test200,None,2024-04-03 02:15:23
1,BA9TJJB3ZA,2024-04-04 05:47:41,Unreturned Equipment,CLOSED,11697715190,WEST,False,2024-04-04 06:47:41,test200,offered credit,2024-04-04 10:47:41
2,XM2O2FWQK6,2024-04-15 06:01:13,Duplicate Charge,CLOSED,311765306940,EAST,False,2024-04-15 11:01:13,test200,offered credit,2024-04-15 14:01:13
3,IQNAQCWW6R,2024-04-11 03:34:44,Billing Error,CLOSED,774886296613,EAST,True,2024-04-11 04:34:44,test200,None,2024-04-11 07:34:44
4,O08WYAIAOA,2024-04-09 18:54:08,Incorrect Package Pricing,CLOSED,994259136122,SOUTH,False,2024-04-09 19:54:08,test200,offered credit,2024-04-09 20:54:08


In [6]:
df_may.head()

,ticketId,createDate,problemSummary,status,accountNumber,region,escalated,frt,assigned_to,resolutionDetails,endtime
0,E2TTXNSO5J,2024-05-17 9:23:00,Unreturned Equipment,CLOSED,166061588498,EAST,True,2024-05-17 14:23:00,test200,offered credit,2024-05-17 18:23:00
1,L3O8GT1K2B,2024-05-28 1:01:35,Incorrect Package Pricing,CLOSED,627006274076,EAST,False,2024-05-28 2:01:35,test200,None,2024-05-28 6:01:35
2,22KRXVFXRF,2024-05-16 16:19:33,Promotion Not Applied,CLOSED,738994699288,SOUTH,False,2024-05-16 18:19:33,test200,offered credit,2024-05-16 20:19:33
3,7TFIQGFVQM,2024-05-29 17:21:13,Billing Error,CLOSED,777001096724,EAST,True,2024-05-29 18:21:13,test200,offered credit,2024-05-29 19:21:13
4,XKSJEMZESO,2024-05-10 6:28:48,Late Payment Fee Dispute,CLOSED,567062102783,WEST,False,2024-05-10 11:28:48,test200,None,2024-05-10 12:28:48


In [7]:
df_april.columns

Index(['ticketId', 'createDate', 'problemSummary', 'status', 'accountNumber',
       'region', 'escalated', 'frt', 'assigned_to', 'resolutionDetails',
       'endtime'],
      dtype='object')

In [8]:
df_may.columns

Index(['ticketId', 'createDate', 'problemSummary', 'status', 'accountNumber',
       'region', 'escalated', 'frt', 'assigned_to', 'resolutionDetails',
       'endtime'],
      dtype='object')

In [9]:
len(df_april)

45

In [10]:
len(df_may)

45

In [11]:
#merge these two dataframes to do analysis on both april and may data - using concat instead of merge to combine vertically
df_april.shape #(45,11)
df_may.shape #(45,11)
pd.concat([df_april,df_may]).shape #expecting: (90,11)

(90, 11)

In [12]:
df_merge=pd.concat([df_april,df_may])

In [13]:
len(df_merge)

90

In [14]:
df_merge['createDate'].max()

'2024-05-30 4:24:10'

In [15]:
df_merge['createDate'].min()

'2024-04-02 04:56:00'

In [16]:
#requested that the final table have timestamp datatypes so need to cast to timestamp, the schema when pulling from APIs
#usually does not auto interpret timestamps

print(df_merge.dtypes)

ticketId             object
createDate           object
problemSummary       object
status               object
accountNumber         int64
region               object
escalated              bool
frt                  object
assigned_to          object
resolutionDetails    object
endtime              object
dtype: object


In [17]:
df_merge['createDate'] = pd.to_datetime(df_merge['createDate'])
df_merge['frt'] = pd.to_datetime(df_merge['frt'])
df_merge['endtime'] = pd.to_datetime(df_merge['endtime'])

In [18]:
print(df_merge.dtypes)

ticketId                     object
createDate           datetime64[ns]
problemSummary               object
status                       object
accountNumber                 int64
region                       object
escalated                      bool
frt                  datetime64[ns]
assigned_to                  object
resolutionDetails            object
endtime              datetime64[ns]
dtype: object


In [19]:
#for the pipeline framework I use at my job, there needs to be an incremental date column for daily journaling
#called incremental which is in UTC and can adjust for daylight savings year round

#first needed to provide tz details to the df for createDate which was confirmed to be captured in EST by the source

df_merge['createDate'] = df_merge['createDate'].dt.tz_localize('US/Eastern')

df_merge['incremental'] = df_merge['createDate'].dt.tz_convert('UTC')

In [20]:
df_merge[['ticketId', 'createDate','incremental']].head()

,ticketId,createDate,incremental
0,KYR361R47B,2024-04-02 18:15:23-04:00,2024-04-02 22:15:23+00:00
1,BA9TJJB3ZA,2024-04-04 05:47:41-04:00,2024-04-04 09:47:41+00:00
2,XM2O2FWQK6,2024-04-15 06:01:13-04:00,2024-04-15 10:01:13+00:00
3,IQNAQCWW6R,2024-04-11 03:34:44-04:00,2024-04-11 07:34:44+00:00
4,O08WYAIAOA,2024-04-09 18:54:08-04:00,2024-04-09 22:54:08+00:00


In [21]:
#need to change the date format and drop the timestamp of incremental in order for our internal tools to be able to
#process daily journaling of this table
df_merge['incremental'] = df_merge['incremental'].dt.strftime('%m-%d-%Y')

In [22]:
df_merge[['ticketId', 'createDate','incremental']].head()

,ticketId,createDate,incremental
0,KYR361R47B,2024-04-02 18:15:23-04:00,04-02-2024
1,BA9TJJB3ZA,2024-04-04 05:47:41-04:00,04-04-2024
2,XM2O2FWQK6,2024-04-15 06:01:13-04:00,04-15-2024
3,IQNAQCWW6R,2024-04-11 03:34:44-04:00,04-11-2024
4,O08WYAIAOA,2024-04-09 18:54:08-04:00,04-09-2024


In [23]:
#the reporting team wanted to be able to look at duration of customer service interactions at xfinity stores,
#so need to add a column that contains duration based on frt and endtime in minutes
df_merge[['ticketId', 'frt','endtime']].sample()

,ticketId,frt,endtime
27,QFCP3B19Y8,2024-05-17 23:05:25,2024-05-18 02:05:25


In [24]:
df_merge['duration'] = (df_merge['endtime'] - df_merge['frt'])

In [25]:
#checking that the duration column from the sample record above has been properly calculated
result = df_merge[df_merge['ticketId'] == 'BHQFJCPQUZ'][['ticketId', 'frt', 'endtime','duration']]
display(result)

,ticketId,frt,endtime,duration
35,BHQFJCPQUZ,2024-05-05 01:34:32,2024-05-05 03:34:32,0 days 02:00:00


In [26]:
#now updating the duration column to contain the duration length in minutes
df_merge['duration'] = df_merge['duration'].dt.total_seconds() / 60

result = df_merge[df_merge['ticketId'] == 'BHQFJCPQUZ'][['ticketId', 'frt', 'endtime','duration']]
display(result)

,ticketId,frt,endtime,duration
35,BHQFJCPQUZ,2024-05-05 01:34:32,2024-05-05 03:34:32,120.0


In [27]:
df_merge.head()

,ticketId,createDate,problemSummary,status,accountNumber,region,escalated,frt,assigned_to,resolutionDetails,endtime,incremental,duration
0,KYR361R47B,2024-04-02 18:15:23-04:00,Late Payment Fee Dispute,CLOSED,282416533844,NORTHEAST,True,2024-04-02 22:15:23,test200,None,2024-04-03 02:15:23,04-02-2024,240.0
1,BA9TJJB3ZA,2024-04-04 05:47:41-04:00,Unreturned Equipment,CLOSED,11697715190,WEST,False,2024-04-04 06:47:41,test200,offered credit,2024-04-04 10:47:41,04-04-2024,240.0
2,XM2O2FWQK6,2024-04-15 06:01:13-04:00,Duplicate Charge,CLOSED,311765306940,EAST,False,2024-04-15 11:01:13,test200,offered credit,2024-04-15 14:01:13,04-15-2024,180.0
3,IQNAQCWW6R,2024-04-11 03:34:44-04:00,Billing Error,CLOSED,774886296613,EAST,True,2024-04-11 04:34:44,test200,None,2024-04-11 07:34:44,04-11-2024,180.0
4,O08WYAIAOA,2024-04-09 18:54:08-04:00,Incorrect Package Pricing,CLOSED,994259136122,SOUTH,False,2024-04-09 19:54:08,test200,offered credit,2024-04-09 20:54:08,04-09-2024,60.0


In [28]:
#also to show conditional logic, have added a new column that flags any records where the customer service interaction
#involved a billing error and the interaction was escalated to a manager
df_merge['billing_escalations'] = np.where(
    (df_merge['problemSummary'] == 'Billing Error') & (df_merge['escalated'] == True), 1, 0
)

In [29]:
df_merge.head()

,ticketId,createDate,problemSummary,status,accountNumber,region,escalated,frt,assigned_to,resolutionDetails,endtime,incremental,duration,billing_escalations
0,KYR361R47B,2024-04-02 18:15:23-04:00,Late Payment Fee Dispute,CLOSED,282416533844,NORTHEAST,True,2024-04-02 22:15:23,test200,None,2024-04-03 02:15:23,04-02-2024,240.0,0
1,BA9TJJB3ZA,2024-04-04 05:47:41-04:00,Unreturned Equipment,CLOSED,11697715190,WEST,False,2024-04-04 06:47:41,test200,offered credit,2024-04-04 10:47:41,04-04-2024,240.0,0
2,XM2O2FWQK6,2024-04-15 06:01:13-04:00,Duplicate Charge,CLOSED,311765306940,EAST,False,2024-04-15 11:01:13,test200,offered credit,2024-04-15 14:01:13,04-15-2024,180.0,0
3,IQNAQCWW6R,2024-04-11 03:34:44-04:00,Billing Error,CLOSED,774886296613,EAST,True,2024-04-11 04:34:44,test200,None,2024-04-11 07:34:44,04-11-2024,180.0,1
4,O08WYAIAOA,2024-04-09 18:54:08-04:00,Incorrect Package Pricing,CLOSED,994259136122,SOUTH,False,2024-04-09 19:54:08,test200,offered credit,2024-04-09 20:54:08,04-09-2024,60.0,0


In [30]:
#normally use sql from the final table in s3 for these types of details, but for this exercise I am showing
#how I would use pandas to get summary stats of different interactions in April and May
duration_frequencies = df_merge['duration'].value_counts()

duration_stats = df_merge['duration'].describe()

total_billing_escalations = df_merge['billing_escalations'].sum()

In [31]:
print(duration_frequencies)

duration
180.0    28
240.0    26
60.0     22
120.0    14
Name: count, dtype: int64


In [32]:
print(duration_stats)

count     90.000000
mean     158.666667
std       68.682628
min       60.000000
25%      120.000000
50%      180.000000
75%      240.000000
max      240.000000
Name: duration, dtype: float64


In [33]:
print(total_billing_escalations)

7


In [ ]:
#in the script I wrote for my job this df would be written into our s3 datalake as a parquet file because we use Spark,
#and then used by BI for reporting. For this exercise I am exporting as CSV to my local Desktop

#check this with Jae: not sure how someone else can see that this is successful so want to test
df_merge.to_csv('/Users/tiadoherty/Desktop/code_sample/final_april_may_df.csv', index=False)